# Creating word vectors using word2vec implementation in Spark MLlib

### Here we show how to use Spark MLlib Word2Vec for generating word-features 
#### The data being used is the attack comments text data

Depeding on the size of your DSVM, this notebook may take about 1-2 mins to finish

MLlib Word2Vec: https://spark.apache.org/docs/2.2.0/mllib-feature-extraction.html#word2vec

## Set directory path for input data 
#### Input data is downloaded locally to a DSVM

In [88]:
# 1. Location of training data on 
text_file = "../Data/text_data/attack_data.csv"

## Set spark context and import necessary libraries

In [89]:
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SQLContext, SparkSession, DataFrame, SparkSession
from pyspark.sql.functions import UserDefinedFunction, regexp_replace, trim, col, lower, lit, udf, monotonically_increasing_id
from pyspark.sql.types import *
from pyspark.ml.feature import Word2Vec, Word2VecModel, Tokenizer, RegexTokenizer, StopWordsRemover
from pyspark.ml.linalg import Vectors

import matplotlib.pyplot as plt
import numpy as np
import datetime
import pandas as pd


sqlContext = SQLContext(sc)

## Data ingestion: Read in attack text csv data

In [93]:
## READ IN DATA AND CREATE SPARK DATAFRAME FROM A CSV
text_df = spark.read.csv(path=text_file, header=True, inferSchema=True, sep=",")
text_df.cache(); text_df.count();

## REGISTER DATA-FRAME AS A TEMP-TABLE IN SQL-CONTEXT
text_df.createOrReplaceTempView("text_table")

text_df.head(1)

[Row(rev_id=37675, comment="`-NEWLINE_TOKENThis is not ``creative``.  Those are the dictionary definitions of the terms ``insurance`` and ``ensurance`` as properly applied to ``destruction``.  If you don't understand that, fine, legitimate criticism, I'll write up ``three man cell`` and ``bounty hunter`` and then it will be easy to understand why ``ensured`` and ``insured`` are different - and why both differ from ``assured``.NEWLINE_TOKENNEWLINE_TOKENThe sentence you quote is absolutely neutral.  You just aren't familiar with the underlying theory of strike-back (e.g. submarines as employed in nuclear warfare) guiding the insurance, nor likely the three man cell structure that kept the IRA from being broken by the British.  If that's my fault, fine, I can fix that to explain.  But ther'es nothing ``personal`` or ``creative`` about it.NEWLINE_TOKENNEWLINE_TOKENI'm tired of arguing with you.  Re: the other article, ``multi-party`` turns up plenty, and there is more use of ``mutually`` t

## Select and filter data set

In [45]:
### SELCT ONLY REV_ID AND COMMENT FIELDS, AND FILTER FOR TRAINING DATA AND ARTILES ONLY
sqlStatement = """ SELECT rev_id, comment 
            FROM text_table 
            where ns = 'article' and split = 'train' """
text_filtered_df = spark.sql(sqlStatement)

## CACHE NEW DATAFRAME IN MEMORY AND CREATE TEMPORARY TABLE
text_filtered_df.cache(); text_filtered_df.count();
text_filtered_df.createOrReplaceTempView("text_filtered_table")

## COUNT NUMBER OF ROWS IN DATAFRAME 
text_filtered_df.count()

31253

## Lowercase and remove some words and remove punctuations

In [98]:
text_filtered_df2 = text_filtered_df.withColumn("comment1", lower(col("comment"))).\
    withColumn("comment2", regexp_replace("comment1", '-newline_token', "")).\
    withColumn("comment3", regexp_replace("comment2", 'newline_token', "")).\
    withColumn("comment_final", regexp_replace("comment3", '[^\w-_ ]', "")).\
    select('rev_id', 'comment_final')
    
text_filtered_df2.head(1)

[Row(rev_id=37675, comment_final='this is not creative  those are the dictionary definitions of the terms insurance and ensurance as properly applied to destruction  if you dont understand that fine legitimate criticism ill write up three man cell and bounty hunter and then it will be easy to understand why ensured and insured are different - and why both differ from assuredthe sentence you quote is absolutely neutral  you just arent familiar with the underlying theory of strike-back eg submarines as employed in nuclear warfare guiding the insurance nor likely the three man cell structure that kept the ira from being broken by the british  if thats my fault fine i can fix that to explain  but theres nothing personal or creative about itim tired of arguing with you  re the other article multi-party turns up plenty and there is more use of mutually than mutual  if i were to apply your standard id be moving mutual assured destruction to talk for not appealing to a reagan voters biases abo

## Tokenize and remove stopwords

In [101]:
## DEFINE TOKENIZER AND STOPWORD REMOVER
tokenizer = Tokenizer(inputCol="comment_final", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="filtWords")

## TRANSFORM DATASET
text_filtered_df3 = tokenizer.transform(text_filtered_df2)
text_filtered_df4 = remover.transform(text_filtered_df3)
text_filtered_df4.cache(); text_filtered_df4.count();
text_filtered_df4.show(5)

+------+--------------------+--------------------+--------------------+
|rev_id|       comment_final|               words|           filtWords|
+------+--------------------+--------------------+--------------------+
| 37675|this is not creat...|[this, is, not, c...|[creative, , dict...|
| 44816| the term standar...|[, the, term, sta...|[, term, standard...|
| 49851|true or false the...|[true, or, false,...|[true, false, sit...|
| 93890|this page will ne...|[this, page, will...|[page, need, disa...|
|103624|i removed the fol...|[i, removed, the,...|[removed, followi...|
+------+--------------------+--------------------+--------------------+
only showing top 5 rows



## DEFINE AND RUN WORD2VEC ON COMMENTS

In [102]:
model = None
window_size = 5
vector_size = 50
min_count = 5

word2Vec = Word2Vec(windowSize = window_size, vectorSize = vector_size, minCount=min_count, numPartitions=2, inputCol="filtWords", outputCol="result")
model = word2Vec.fit(text_filtered_df4)

## Examine some words, and other words close to them from these feature neighborhood

In [103]:
model.findSynonyms("bad", 20).select("word").head(5)

[Row(word='assume'),
 Row(word='wikipediaassume'),
 Row(word='good'),
 Row(word='luck'),
 Row(word='positively')]

## Examine how vector features look like

In [105]:
word2vec_features = model.getVectors().select("*")
word2vec_features.head(1)

[Row(word='quotient', vector=DenseVector([-0.0403, -0.0233, -0.0682, -0.0333, -0.0461, 0.0497, -0.0433, -0.0738, 0.014, 0.0308, -0.0249, -0.0422, -0.0002, 0.0057, -0.009, 0.0117, -0.0029, 0.0486, 0.0122, -0.0006, -0.0098, 0.0192, 0.0119, -0.0721, 0.0346, -0.0509, 0.0527, 0.0309, 0.0387, -0.1122, -0.0355, 0.0262, 0.044, 0.05, -0.0241, 0.0363, -0.018, 0.0516, 0.022, 0.0522, -0.0461, -0.0216, -0.0054, -0.0932, 0.0132, 0.0156, 0.0199, 0.0676, -0.0154, -0.0674]))]

## Convert Spark DF to Pandas DF

In [85]:
word2vec_features_pdf = word2vec_features.toPandas()
word2vec_features_pdf.head(4)

,word,vector
0,quotient,"[-0.0164144989103, -0.0146850179881, -0.039069..."
1,incident,"[-0.081671461463, 0.0413153469563, -0.01991129..."
2,serious,"[-0.16352725029, 0.079047486186, -0.0693107396..."
3,wgbh,"[-0.0464854203165, 0.0286101084203, -0.0830683..."


## Save features in CSV for subsequent steps

In [87]:
word2vec_features_pdf.to_csv('../Outputs/Word2Vec-Features.csv')

## Getting comment-level vectors from word-level vectors (averaging)

In [115]:
comment_vectors_df = model.transform(text_filtered_df4).select('rev_id','result')
comment_vectors_df.head(2)

[Row(rev_id=37675, result=DenseVector([0.0302, 0.1256, -0.0132, -0.0225, 0.032, -0.0435, 0.0599, 0.1015, 0.1214, -0.0662, -0.0778, 0.1033, 0.0209, 0.0176, -0.0662, 0.0338, 0.0508, -0.068, 0.0734, -0.0304, -0.0309, -0.1231, -0.0275, 0.062, 0.0069, 0.056, -0.0824, -0.1513, -0.0476, 0.056, -0.0014, -0.029, -0.0252, -0.1051, 0.1535, -0.0756, -0.0618, -0.1073, -0.0373, -0.0182, -0.0479, 0.1109, 0.0215, 0.0355, -0.0246, -0.0338, -0.0695, 0.0414, -0.0908, -0.0566])),
 Row(rev_id=44816, result=DenseVector([0.0324, 0.1015, -0.0437, -0.021, 0.0139, -0.0662, 0.0401, 0.1435, 0.1537, -0.0643, -0.0939, 0.0719, -0.0106, 0.0014, -0.1009, 0.013, 0.1032, -0.0698, 0.0952, 0.0003, 0.0314, -0.132, -0.0417, 0.0768, -0.0222, 0.0495, -0.0754, -0.1567, -0.0258, 0.1161, -0.0214, 0.025, 0.0127, -0.1021, 0.1115, -0.066, -0.0902, -0.1024, 0.0042, -0.0361, -0.0994, 0.0256, 0.0491, 0.0538, 0.0126, -0.034, -0.086, 0.0659, -0.131, -0.0832]))]